In [1]:
import tensorflow as tf
from keras.applications import MobileNet
from keras.models import Sequential,Model 
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import keras

In [2]:
# MobileNet is designed to work with images of dim 224,224
img_rows,img_cols = 224,224

MobileNet = MobileNet(weights=None,include_top=False,input_shape=(img_rows,img_cols,3))

for layer in MobileNet.layers:
    layer.trainable = True

# Let's print our layers
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i),layer.__class__.__name__,layer.trainable)

0 InputLayer True
1 ZeroPadding2D True
2 Conv2D True
3 BatchNormalization True
4 ReLU True
5 DepthwiseConv2D True
6 BatchNormalization True
7 ReLU True
8 Conv2D True
9 BatchNormalization True
10 ReLU True
11 ZeroPadding2D True
12 DepthwiseConv2D True
13 BatchNormalization True
14 ReLU True
15 Conv2D True
16 BatchNormalization True
17 ReLU True
18 DepthwiseConv2D True
19 BatchNormalization True
20 ReLU True
21 Conv2D True
22 BatchNormalization True
23 ReLU True
24 ZeroPadding2D True
25 DepthwiseConv2D True
26 BatchNormalization True
27 ReLU True
28 Conv2D True
29 BatchNormalization True
30 ReLU True
31 DepthwiseConv2D True
32 BatchNormalization True
33 ReLU True
34 Conv2D True
35 BatchNormalization True
36 ReLU True
37 ZeroPadding2D True
38 DepthwiseConv2D True
39 BatchNormalization True
40 ReLU True
41 Conv2D True
42 BatchNormalization True
43 ReLU True
44 DepthwiseConv2D True
45 BatchNormalization True
46 ReLU True
47 Conv2D True
48 BatchNormalization True
49 ReLU True
50 DepthwiseCon

In [3]:
def addTopModelMobileNet(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    
    top_model = Dense(1024,activation='relu')(top_model)
    
    top_model = Dense(512,activation='relu')(top_model)
    
    top_model = Dense(num_classes,activation='softmax')(top_model)

    return top_model

num_classes = 3  # ['COVID19','NORMAL','Viral Pneumonia']

FC_Head = addTopModelMobileNet(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)     

In [4]:
train_data_dir = r'D:\Dev65\python_workspace2\test\collection-re2\stacking'
validation_data_dir = r'D:\Dev65\python_workspace2\test\collection-re2\fuzzyColor'

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=15,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    horizontal_flip=True,
                    fill_mode='nearest'
                                   )
print(type(train_datagen))
validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 16

<class 'tensorflow.python.keras.preprocessing.image.ImageDataGenerator'>


In [5]:

train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size = (img_rows,img_cols),
                        batch_size = batch_size,
                        class_mode = 'categorical'
                        )

# 경로를 통해 labeling 한 데이터 셋을 만드는 것
validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical')


Found 3036 images belonging to 3 classes.
Found 3036 images belonging to 3 classes.


In [6]:
from keras.optimizers import RMSprop,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [7]:


checkpoint = ModelCheckpoint(
                             'mobileNetV2model7.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

# earlystop = EarlyStopping(
#                           monitor='val_loss',
#                           min_delta=0,
#                           patience=10,
#                           verbose=1,restore_best_weights=True)

# learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
#                                             patience=5, 
#                                             verbose=1, 
#                                             factor=0.2, 
#                                             min_lr=0.0001)

#callbacks = [earlystop,checkpoint,learning_rate_reduction]
callbacks = [checkpoint]


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )

nb_train_samples = 3036 
nb_validation_samples = 3036 

epochs = 50

history = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,     
            epochs=epochs,
            callbacks=callbacks,
            #validation_split=0.2
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size
)

print("끝났다!")

model.save('mobileNetV2model7_complete.h5')
model.save_weights('mobileNetV2model7_weight.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
189/189 [==============================] - ETA: 0s - loss: 0.6356 - accuracy: 0.7642
Epoch 00001: val_loss improved from inf to 1.26335, saving model to mobileNetV2model7.h5
189/189 [==============================] - 235s 1s/step - loss: 0.6356 - accuracy: 0.7642 - val_loss: 1.2634 - val_accuracy: 0.4438
Epoch 2/50
189/189 [==============================] - ETA: 0s - loss: 0.4193 - accuracy: 0.8566
Epoch 00002: val_loss did not improve from 1.26335
189/189 [==============================] - 234s 1s/step - loss: 0.4193 - accuracy: 0.8566 - val_loss: 2.5054 - val_accuracy: 0.1154
Epoch 3/50
189/189 [==============================] - ETA: 0s - loss: 0.3327 - accuracy: 0.8858
Epoch 00003: val_loss did not improve from 1.26335
189/189 [==============================] - 234s 1s/step - loss: 0.3327 - accuracy: 0.8858 - val_loss: 1.7433 - val_accuracy: 0.1154
Epoch 4/50
189/189 [=============================

Epoch 29/50
189/189 [==============================] - ETA: 0s - loss: 0.1121 - accuracy: 0.9626
Epoch 00029: val_loss did not improve from 0.22769
189/189 [==============================] - 233s 1s/step - loss: 0.1121 - accuracy: 0.9626 - val_loss: 1.1986 - val_accuracy: 0.5592
Epoch 30/50
189/189 [==============================] - ETA: 0s - loss: 0.1201 - accuracy: 0.9560
Epoch 00030: val_loss improved from 0.22769 to 0.16586, saving model to mobileNetV2model7.h5
189/189 [==============================] - 233s 1s/step - loss: 0.1201 - accuracy: 0.9560 - val_loss: 0.1659 - val_accuracy: 0.9438
Epoch 31/50
189/189 [==============================] - ETA: 0s - loss: 0.1133 - accuracy: 0.9636
Epoch 00031: val_loss did not improve from 0.16586
189/189 [==============================] - 232s 1s/step - loss: 0.1133 - accuracy: 0.9636 - val_loss: 0.2512 - val_accuracy: 0.9041
Epoch 32/50
189/189 [==============================] - ETA: 0s - loss: 0.1167 - accuracy: 0.9672
Epoch 00032: val_loss